In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

print("Libraries successfully imported.")

Libraries successfully imported.


In [8]:
data = pd.read_csv("data/train.csv")
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')